In [1]:
from catboost import Pool,CatBoostClassifier
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold , KFold
import matplotlib.pyplot as plt
import numpy as np
import optuna
import lightgbm as lgbm
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from typing import List ,Dict, Tuple
from sklearn.metrics import log_loss
import lightgbm as lgbm
from optuna import trial
from optuna.integration import LightGBMPruningCallback
import seaborn as sns
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot
from optuna.trial import Trial
DATA_PATH = './JobCare_data/'
train = pd.read_csv('./JobCare_data/train.csv')
code_d = pd.read_csv(f'{DATA_PATH}속성_D_코드.csv').iloc[:,:-1]
code_h = pd.read_csv(f'{DATA_PATH}속성_H_코드.csv')
code_l = pd.read_csv(f'{DATA_PATH}속성_L_코드.csv')
train_data = pd.read_csv(f'{DATA_PATH}train.csv')
test_data = pd.read_csv(f'{DATA_PATH}test.csv')

In [81]:
nominal_cols = ['person_attribute_a_1','person_attribute_b','person_prefer_e','contents_attribute_e']
code_d.columns  = ["attribute_d", "attribute_d_d" , "attribute_d_s", "attribute_d_m" , "attribute_d_l"]
code_h.columns = ["attribute_h" , "attribute_h_p"]
code_l.columns = ["attribute_l", "attribute_l_d", "attribute_l_s" , "attribute_l_m", "attribute_l_l"]
ordinal_cols = ['person_attribute_a_1','person_attribute_b', 'person_prefer_e', 'contents_attribute_e']
# 오버피팅이 자주 발생하는 mean encoding 이므로 cross validation 과 정규화 같이 사용한다=
# 변환 하고자 하는 범주형 변수 선택
# 범주형 변수 그룹화 -> 타깃 변수 총합 합계
# 범주형 변수 그룹화 타깃 빈도수 합계
# 총합을 카운트로 나누고 본래 범주 값에 업데이트
# 여러가지 방법으로 적용 가능하다
# 비슷한 범주 사이에 있는 관계 표현 특징, 범주와 타깃사이에만 국한된다
# 범주가 많은 경우 이 방법은 데이터를 훨씬 더 단순화 한다
 #%%
cols_merge = [
              ("person_prefer_d_1" , code_d),
              ("person_prefer_d_2" , code_d),
              ("person_prefer_d_3" , code_d),
              ("contents_attribute_d" , code_d),
              ("person_prefer_h_1" , code_h),
              ("person_prefer_h_2" , code_h),
              ("person_prefer_h_3" , code_h),
              ("contents_attribute_h" , code_h),
              ("contents_attribute_l" , code_l),
]

In [ ]:
# 회원 속성과 콘텐츠 속성의 동일한 코드 여부에 대한 컬럼명 리스트
cols_equi = [
    ("contents_attribute_c","person_prefer_c"),
    ("contents_attribute_e","person_prefer_e"),
    ("person_prefer_d_2_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_2_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_2_attribute_d_l" , "contents_attribute_d_attribute_d_l"),
    ("person_prefer_d_3_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_3_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_3_attribute_d_l" , "contents_attribute_d_attribute_d_l"),
    ("person_prefer_h_1_attribute_h_p" , "contents_attribute_h_attribute_h_p"),
    ("person_prefer_h_2_attribute_h_p" , "contents_attribute_h_attribute_h_p"),
    ("person_prefer_h_3_attribute_h_p" , "contents_attribute_h_attribute_h_p"),
]

In [82]:
train_data.drop(['id','contents_open_dt','contents_rn','person_rn'],axis=1)

In [ ]:

def merge_codes(df:pd.DataFrame, df_code:pd.DataFrame, col:str) -> pd.DataFrame:
    df = df.copy()
    df_code = df_code.copy()
    df_code = df_code.add_prefix(f"{col}_")
    # join key 설정
    df_code.columns.values[0] = col
    print('df :' ,df)
    print('df_code : ', df_code)
    print('merge data : ',pd.merge(df,df_code , how='left', on= col) )
    return pd.merge(df,df_code , how='left', on= col)

def preprocess_data(
                    df:pd.DataFrame,is_train:bool = True,
                    cols_merge:List[Tuple[str,pd.DataFrame]] = []  ,
                    cols_equi:List[Tuple[str,str]]= [] ,
                    cols_drop:List[str] = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt"]
                    )->Tuple[pd.DataFrame,np.ndarray]:

    y_data = None

    if is_train:
        y_data = df["target"].to_numpy()
        df = df.drop(columns="target")

    for col , df_code in cols_merge:
        df = merge_codes(df, df_code, col)

    cols = df.select_dtypes(bool).columns.tolist()
    df[cols] = df[cols].astype(int)

    for col1, col2 in cols_equi:
        df[f"{col1}_{col2}"] = (df[col1] == df[col2]).astype(int)
    df = df.drop(columns=cols_drop)

    return (df,y_data)


def get_meta_data(train_data):
    cols = train_data.columns
    temp =[]
    binary_col = []
    nominal_col = []
    ordinal_col = []
    for col in cols:
        if 'yn' in col:
            temp.append((col,'binary'))
        elif col in ordinal_cols:
            temp.append((col,'ordinal'))
        else:
            temp.append((col,'nominal'))
            
    temp = pd.DataFrame(data)
    
    temp.columns = ['col_name', 'type']
    return temp

In [ ]:
def get_mean_encode(train, nominal_cols):
    for col in nominal_cols:
        if 'attribute' in col:
            name = col[col.index('attribute') + len('attribute')+1:]
            var_name = 'attr_{}_mean_encode'.format(name)
            locals()[var_name] = train.groupby(col)["target"].mean()
        elif 'prefer' in col:
            name = col[col.index('prefer') + len('prefer')+1:]
            var_name = 'prefer_{}_mean_encode'.format(name)
            locals()[var_name] = train.groupby(col)["target"].mean()
        train.loc[:,var_name] = train[col].map(locals()[var_name])
    return train


def objective(trial,X,y):
    param_grid = {
        # "device_type" : trial.suggest_categorical("device_type",['gpu']),
        "learning_rate" : trial.suggest_float("learning_rate", 0.1, 0.95, step=0.1 ),
        "num_leaves" : trial.suggest_int("num_leaves", 5, 100 ,step=5),
        "max_depth" : trial.suggest_int("max_depth", 1,30 , step=3),
        "min_data_in_leaf" : trial.suggest_int("min_data_in_leaf" ,50 , 1000 , step=50),
        "lambda_l1" : trial.suggest_int("lambda_l1", 0,100 , step=5),
        "lambda_l2" : trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split" : trial.suggest_float("min_datin_to_split" , 1,30, step=3),
        "bagging_fraction" : trial.suggest_float("bagging_fraction" , 0.1,0.99,step=0.1),
        "bagging_freq" : trial.suggest_categorical("bagging_freq",[1]),
        "feature_fraction" : trial.suggest_float("feature_fraction", 0.1,0.99,step=0.1),
        "n_estimators" : trial.suggest_int("n_estimators",10,2000, step=10),
    }
    cv=  StratifiedKFold(n_splits=5, shuffle=False)
    cv_scores = np.empty(5)

    for idx, (train_idx, test_idx ) in enumerate(cv.split(X,y)):
        X_train , X_test = X.iloc[train_idx] , X.iloc[test_idx]
        y_train , y_test = y[train_idx] , y[test_idx]
        model = lgbm.LGBMClassifier(objective="binary" , **param_grid)

        model.fit(
            X_train,
            y_train,
            eval_set = [(X_test, y_test)],
            eval_metric = "binary_logloss",
            early_stopping_rounds =100,
            callbacks= [
                LightGBMPruningCallback(trial, "binary_logloss")
            ], # Add a pruning callback
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test,preds)

    #return np.mean(f1_scores)
    return np.mean(cv_scores)

In [ ]:
# :::::::::::: META DATA MAKE :::::::::::::::::::
data = get_meta_data(train_data) 
# :::::::::::: META DATA MAKE END :::::::::::::::::::
# get mean_encdoe train set
train = get_mean_encode(train, nominal_cols)
target = train.target
# drop target , contetns_open_dt, id
train = train.drop(['contents_open_dt','id','target'],axis=1)
x_train, x_valid, y_train, y_valid = train_test_split(train,target , test_size=0.3 , shuffle=True, stratify=target, random_state=34)
# gradient boosting 이 randomforest 보다 좋다고 알려져있다
rf= RandomForestClassifier(n_estimators=150 , max_depth=8 , min_samples_split=4, max_features=0.2, n_jobs=-1 ,random_state=0)
rf.fit(x_train,y_train)

In [ ]:
# :::::::::::: HYPER PARAMETER OPTUNA START  LGBM CLASSIFIER:::::::::::::::::::::::

# mean encode값 랜포보다 성능좋다고 알려진 Lightgbm에 넣기
study = optuna.create_study(direction="minimize" , study_name = "LGBM Classifier")
func = lambda trial: objective(trial,train,target )
study.optimize(func,n_trials=20)
Trial.suggest_float(name="learning_rate",low= 0.1 ,high=0.95, step=0.1 )
trial = Trial()
trial.suggest_float(name="learning_rate",low= 0.1 ,high=0.95, step=0.1 )
# trial suggest_float 메서드는 self 가 필요한데 이렇게 익명함수 명을 함수명으로 해서 다른 함수의 인자로 넘기면 가능한가 ?
train = train.drop(['person_rn','contents_rn'], axis=1)
# 과적합일진 모르지만 그래도많이올랐따 만족할 수 없음
# 0.57정도에서 0.59 정도로 mean encoding 으로 상승 -> 랜포말고 다른건 ?

In [73]:
# 시작과 끝을 잘 경계짓고 , 범주를 잘 정리 변수의 위치 등

In [ ]:
#define dataset
y = y_train.values.tolist()
X, y = make_regression(n_samples=1000, n_features=10 , n_informative=5, random_state=1)
#evaluate the model
model = CatBoostRegressor(verbose=0 , n_estimators=100)
cv = RepeatedKFold(n_splits= 10 , n_repeats=3 , random_state=1)
n_scores = cross_val_score(model, X,y , scoring='neg_mean_absolute_error', cv=cv)
print('MAE : %.3f (%.3f)'  % (mean(n_scores), std(n_scores)))
#fit the model on the whole dataset
model.fit(X,y)
# make a single prediction
row = x_valid[[col for col in x_train.columns if 'mean_encode' in col]].values.tolist()
#row = [[2.02220122, 0.31563495, 0.82797464, -0.30620401, 0.16003707, -1.44411381, 0.87616892, -0.50446586, 0.23009474, 0.76201118]]
# make a single prediction
yhat = model.predict(row)
print('Prediction: %.3f' % yhat[0])
(yhat == y_valid.values.tolist()).mean()
#::::::::::::: CATBOOSTREGRESSOR END :::::::::::::::::::::#
#catboost + cv 5fold + threshold

In [1]:
#Backward difference 코딩에서
# #수준에 대한 종속 변수의 평균은 이전 수준에 대한 종속 변수의 평균과 비교됩니다. 이러한 유형의 코딩은 명목 또는 순서 변수에 유용할 수 있습니다.'
#In backward difference coding,
# the mean of the dependent variable for a level is
# compared with the mean of the dependent variable for the prior level.
# This type of coding may be useful for a nominal or an ordinal variable.'

#::::::::::::: SEABORN DATA PLOT START::::::::::::::::#
fig, ax = plt.subplots(5,1,figsize=(10,20))
for i in range(len(ordinal_cols)):
    print(train_data[ordinal_cols[i]].value_counts())
    sns.histplot(train_data[ordinal_cols[i]].values ,ax=ax[i])
#::::::::::::: SEABORN DATA PLOT END::::::::::::::::#

# 하나는 있는 거니까 , left merge는 있는 컬럼키면 그대로두고
# 없는 컬럼 키면 그대로 붙이니까 merge가 가능하다

In [ ]:
#:::::::::::::

List[Tuple[str,pd.DataFrame]]
# type hinting , type annotation 타입에 대한 메타 정보를 제공한다 중대규모 이상프로젝트에서는 컴파일 위험 에러
 

# 학습에 필요없는 컬럼 리스트
cols_drop = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt", "contents_rn", ]
x_train, y_train = preprocess_data(train_data, cols_merge = cols_merge , cols_equi= cols_equi , cols_drop = cols_drop)
x_test, _ = preprocess_data(test_data,is_train = False, cols_merge = cols_merge , cols_equi= cols_equi  , cols_drop = cols_drop)
x_train.shape , y_train.shape , x_test.shape
x_train  = x_train.iloc[:351365,:]
y_train = y_train[:351365]

In [ ]:
mean_encode = x_train[[col for col in x_train.columns if 'mean_encode' in col]]

In [ ]:
#left는 그래도 on key 가 하나는 있어야 없어도 붙인다
x_train
mean_encode =  mean_encode.reset_index(drop=True)
x_train = pd.concat([x_train, mean_encode],axis=1 )
x_train.columns[x_train.nunique() > 2 ].tolist()
# 범주형 컬럼 리스트

In [ ]:
x_train
is_holdout = False
n_splits = 5
iterations = 3000
patience= 50
# 전역 변수
scores = []
models = []
# category feature 가 아니지만 쓰였으므로
cat_features = x_train.columns[x_train.nunique() > 2].tolist()
SEED = 42 
cv = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
models = []
for tri, vai in cv.split(x_train):
    print("="*50)
    preds = []
    model = CatBoostClassifier(iterations=iterations,random_state=SEED,eval_metric="F1",cat_features=cat_features,one_hot_max_size=4)
    model.fit(x_train.iloc[tri], y_train[tri],
            eval_set=[(x_train.iloc[vai], y_train[vai])],
            early_stopping_rounds=patience ,
            verbose = 100
        )
    models.append(model)
    scores.append(model.get_best_score()["validation"]["F1"])
    if is_holdout:
        break

model = tf.keras.Sequential([
                tf.keras.layers.Dense(units=48, activation = 'relu' , input_shape=(68,)),
                tf.keras.layers.Dense(units=24, activation = 'relu'),
                tf.keras.layers.Dense(units=12, activation = 'relu'),
                #마지막 레이어 유닛은 1로 해야 계산을 할 수 가 있다
                tf.keras.layers.Dense(units=1 , activation = 'sigmoid')
])
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.07),
              loss='binary_crossentropy' ,metrics=['accuracy'])
 
model.summary()    
history = model.fit(x_train , y_train , epochs=25, batch_size = 32, validation_split=0.25, callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')])
history

In [ ]:
 
# train start
  
#cat_features 를 지정하지 않아도

In [ ]:
# 모델에 맞지 않는 피쳐값을 넣으니 성능이 하락하네 이런 이유가 있을 수 있다
 
# 피쳐별 가중치를 달리하는 딥러닝 모델 훈련 ? 이 의미가 있을까싶은데 내가 원하는 피쳐별 가중치를 달리하게 해주는 것은 딥러닝,용도와 의미를 이제 깨달았다
 
